In [2]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [8]:
# connection to gans database in cloud (AWS)

schema = 'gans'
host = 'wbs-project4-db.cdf7pal02v3u.eu-central-1.rds.amazonaws.com'
user = 'admin'
password = 'Your RDS Instance password'
port = 3306
con = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"

In [ ]:
# city_data is extracted from sql db as it's static data, also enables merge to get city id
# comment out unless testing function

'''airport_data = pd.read_sql_table(
    'airports', con=con, index_col=None, coerce_float=True, parse_dates=None, columns=None, chunksize=None)
icao = airport_data['icao'].to_list()'''

In [5]:
icao = airport_data['icao'].to_list()
#icao

## API Data Structue

this is the structure of the data returned by the API when formatted

{"arrivals":[
	{"departure":{
		"airport":{
			"icao":"LGMK","iata":"JMK","name":"Mykonos Island"
			},
		"quality":[]
			},
		"arrival":{
			"scheduledTimeLocal":"2022-07-08 20:05+02:00",
            "actualTimeLocal":"2022-07-08 20:05+02:00",
            "scheduledTimeUtc":"2022-07-08 18:05Z",
            "actualTimeUtc":"2022-07-08 18:05Z",
            "terminal":"1",
            "quality":["Basic","Live"]
			},
	"number":"A3 1543",
	"status":"Expected",
	"codeshareStatus":"IsCodeshared",
	"isCargo":false,
	"aircraft":{
           "reg":"D-AIJC",
           "modeS":"3C6543",
           "model":"Airbus A320 NEO"
           },
	"airline":{"name":"Aegean"}
	},
	{"departure":{"airport":{"icao": ...... and so on

In [24]:
# function is required to enable auto updating in the cloud once linked to sql
# this function can now be copied to the Lambda_handler function in AWS

def tomorrows_flight_arrivals(icao_list):
    
    # city_data is extracted from sql db as it's static data, also enables merge to get city id
    airport_data = pd.read_sql_table(
        'airports', con=con, index_col=None, coerce_float=True, parse_dates=None, columns=None, chunksize=None)
    icao = airport_data['icao'].to_list()
    
    # these are the times to retrieve data for next day
today = datetime.now().astimezone(timezone('Europe/Berlin')).date() # these are the times to retrieve data for next day
    tomorrow = (today + timedelta(days=1))
    
    list_for_df =[]
    
    for icao in icao_list:
        times = [['00:00','11:59'],['12:00','23:59']] # data can be called from API for max of 12hrs so x2 periods defined per day
        
        for t in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{t[0]}/{tomorrow}T{t[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCargo":"false","withPrivate":"false"}
            headers = {
                "X-RapidAPI-Key": "Use Your API key here",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
        r = requests.request('GET', url, headers=headers, params=querystring)
        flights_json = r.json()
        
        for flight in flights_json['arrivals']:    # arrivals is the 1st item returned by API - this dict contains all other dicts/lists
            flights_dict = {}                     # blank dict created to hold extracted values created below
            flights_dict['arrival_icao'] = icao   # this is the icao value returned from the API list ref'd through the for loop 
            # .get() is another way of ensuring our code doesn't break
            # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
            # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
            flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)   # various cols formed from the dict & list refs under 'arrivals'
            flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
            flights_dict['status'] = flight.get('status', None)
            flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
            flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
            flights_dict['airline'] = flight['airline'].get('name', None)
            flights_dict['flight_number'] = flight.get('number', None)    # here number is directly under arrivals 
            flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
            list_for_df.append(flights_dict) # the list_for_df is appended with values created, this enables pandas to convert easily to a dataframe
            
        arrivals_data = pd.DataFrame(list_for_df) # converting list to dataframe
        arrivals_data["arrival_time_local"] = pd.to_datetime(arrivals_data["arrival_time_local"])
            
    return arrivals_data # drop return when using in lambda function AWS
        

In [51]:
# test function using the earlier defined icao list from the imported airport_data

tomorrows_flight_arrivals(icao).head()

In [57]:
arrivals_data.dtypes

arrival_icao                                         object
arrival_time_local    datetime64[ns, pytz.FixedOffset(120)]
arrival_terminal                                     object
departure_city                                       object
departure_icao                                       object
airline                                              object
flight_number                                        object
data_retrieved_on                                    object
dtype: object